# Aggregation Lab

## Group: &lt; Lab Group Number here &gt;

### Members: &lt; Members of the group here &gt; 

We have discussed **data**, **marks**, **encodings**, and **encoding types**.
Another essential piece of Altair's API is its approach to binning and aggregating data

In [1]:
import altair as alt

In [2]:
from vega_datasets import data
cars = data.cars()

cars.head()

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA


### Group-By in Pandas

One key operation in data exploration is the *group-by*, discussed in detail in [Chaper 4](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html) of the *Python Data Science Handbook*.
In short, the group-by *splits* the data according to some condition, *applies* some aggregation within those groups, and then *combines* the data back together:

![Split Apply Combine figure](split-apply-combine.png)
[Figure source](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html)

For the cars data, you might split by Origin, compute the mean of the miles per gallon, and then combine the results.
In Pandas, the operation looks like this:

In [3]:
cars.groupby('Origin')['Miles_per_Gallon'].mean()

Origin
Europe    27.891429
Japan     30.450633
USA       20.083534
Name: Miles_per_Gallon, dtype: float64

In Altair, this sort of split-apply-combine can be performed by passing an aggregation operator within a string to any encoding. For example, we can display a plot representing the above aggregation as follows:

In [4]:
alt.Chart(cars).mark_bar().encode(
    y='Origin',
    x='mean(Miles_per_Gallon)'
)

alt.Chart(...)

Notice that the grouping is done implicitly within the encodings: here we group only by Origin, then compute the mean over each group.

### One-dimensional Binnings: Histograms

One of the most common uses of binning is the creation of histograms. For example, here is a histogram of miles per gallon:

In [5]:
alt.Chart(cars).mark_bar().encode(
    alt.X('Miles_per_Gallon', bin=True),
    alt.Y('count()'),
    alt.Color('Origin')
)

alt.Chart(...)

One interesting thing that Altair's declarative approach allows us to start assigning these values to different encodings, to see other views of the exact same data.

So, for example, if we assign the binned miles per gallon to the color, we get this view of the data:

In [6]:
alt.Chart(cars).mark_bar().encode(
    color=alt.Color('Miles_per_Gallon', bin=True),
    x='count()',
    y='Origin'
)

alt.Chart(...)

This gives us a better appreciation of the proportion of MPG *within* each country.

If we wish, we can normalize the counts on the x-axis to compare proportions directly:

In [7]:
alt.Chart(cars).mark_bar().encode(
    color=alt.Color('Miles_per_Gallon', bin=True),
    x=alt.X('count()', stack='normalize'),
    y='Origin'
)

alt.Chart(...)

We see that well over half of US cars were in the "low mileage" category.

Changing the encoding again, let's map the color to the count instead:

In [8]:
alt.Chart(cars).mark_rect().encode(
    x=alt.X('Miles_per_Gallon', bin=alt.Bin(maxbins=20)),
    color='count()',
    y='Origin',
)

alt.Chart(...)

Now we see the same dataset as a heat map!

This is one of the beautiful things about Altair: it shows you through its API grammar the relationships between different chart types: for example, a 2D heatmap encodes the same data as a stacked histogram!

### Other aggregates

Aggregates can also be used with data that is only implicitly binned.
For example, look at this plot of MPG over time:

In [9]:
alt.Chart(cars).mark_point().encode(
    x='Year:T',
    color='Origin',
    y='Miles_per_Gallon'
)

alt.Chart(...)

The fact that the points overlap so much makes it difficult to see important parts of the data; we can make it clearer by plotting the mean in each group (here, the mean of each Year/Country combination):

In [10]:
alt.Chart(cars).mark_line().encode(
    x='Year:T',
    color='Origin',
    y='mean(Miles_per_Gallon)'
)

alt.Chart(...)

The ``mean`` aggregate only tells part of the story, though: Altair also provides built-in tools to compute the lower and upper bounds of confidence intervals on the mean.

We can use ``mark_area()`` here, and specify the lower and upper bounds of the area using ``y`` and ``y2``:

In [11]:
alt.Chart(cars).mark_area(opacity=0.3).encode(
    x='Year:T',
    color='Origin',
    y='ci0(Miles_per_Gallon)',
    y2='ci1(Miles_per_Gallon)'
)

alt.Chart(...)

## Time Binnings

One special kind of binning is the grouping of temporal values by aspects of the date: for example, month of year, or day of months.
To explore this, let's look at a simple dataset consisting of average temperatures in Seattle:

In [12]:
temps = data.seattle_temps()
temps.head()

,date,temp
0,2010-01-01 00:00:00,39.4
1,2010-01-01 01:00:00,39.2
2,2010-01-01 02:00:00,39.0
3,2010-01-01 03:00:00,38.9
4,2010-01-01 04:00:00,38.8


If we try to plot this data with Altair, we will get a ``MaxRowsError``:

In [13]:
alt.Chart(temps).mark_line().encode(
    x='date:T',
    y='temp:Q'
)

MaxRowsError: ignored

alt.Chart(...)

In [14]:
len(temps)

8759

### Aside: How Altair Encodes Data

We chose to raise a MaxRowsError for datasets larger than 5000 rows because of our observation of students using Altair, because unless you think about how your data is being represented, it's quite easy to end up with **very** large notebooks inwhich performance will suffer.

When you pass a pandas dataframe to an Altair chart, the result is that the data is converted to JSON and stored in the chart specification. This specification is then embedded in the output of your notebook, and if you make a few dozen charts this way with a large enough dataset, it can significantly slow down your machine.

So how to get around the error? A few ways:

1) Use a smaller dataset. For example, we could use Pandas to aggregate the temperatures by day:
   ```python
   import pandas as pd
   temps = temps.groupby(pd.DatetimeIndex(temps.date).date).mean().reset_index()
   ```

2) Disable the MaxRowsError using
   ```python
   alt.data_transformers.enable('default', max_rows=None)
   ```
   But note this can lead to *very* large notebooks if you're not careful.
   
3) Serve your data from a local threaded server. The [altair data server](https://github.com/altair-viz/altair_data_server) package makes this easy. 
   ```python
   alt.data_transformers.enable('data_server')
   ```
   Note that this approach may not work on some cloud-based Jupyter notebook services.
   
4) Use a URL which points to the data source. Creating a [gist](gist.github.com) is a quick and easy way to store frequently used data. 


We'll do the latter here, which is the most convenient and leads to the best performance. All of the sources in `vega_datasets` contain a `url` property. 

In [15]:
temps = data.seattle_temps.url

In [16]:
alt.Chart(temps).mark_line().to_dict()

{'$schema': 'https://vega.github.io/schema/vega-lite/v4.17.0.json',
 'config': {'view': {'continuousHeight': 300, 'continuousWidth': 400}},
 'data': {'url': 'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/seattle-temps.csv'},
 'mark': 'line'}

Notice that instead of including the entire dataset only the url is used.

Now lets try again with our plot

In [17]:
alt.Chart(temps).mark_line().encode(
    x='date:T',
    y='temp:Q'
)

alt.Chart(...)

This data is a little bit crowded; suppose we would like to bin this data by month. We'll do this using ``TimeUnit Transform`` on the date:

In [18]:
alt.Chart(temps).mark_point().encode(
    x=alt.X('month(date):T'),
    y='temp:Q'
)

alt.Chart(...)

This might be clearer if we now aggregate the temperatures:

In [19]:
alt.Chart(temps).mark_bar().encode(
    x=alt.X('month(date):O'),
    y='mean(temp):Q'
)

alt.Chart(...)

We can also split dates two different ways to produce interesting views of the data; for example:

In [20]:
alt.Chart(temps).mark_rect().encode(
    x=alt.X('date(date):O'),
    y=alt.Y('month(date):O'),
    color='mean(temp):Q'
)

alt.Chart(...)

Or we can look at the hourly average temperature as a function of month:

In [21]:
alt.Chart(temps).mark_rect().encode(
    x=alt.X('hours(date):O'),
    y=alt.Y('month(date):O'),
    color='mean(temp):Q'
)

alt.Chart(...)

This kind of transform can be quite useful when working with temporal data.

More information on the ``TimeUnit Transform`` is available here: https://altair-viz.github.io/user_guide/transform/timeunit.html#user-guide-timeunit-transform

## Your Turn!

   1. Plot an area graph showing the min and max of the temperatures in Seattle by month.  Note, we are using the data as loaded above which is via URL so you must specify the data type.
   2. Show a horizontal normalized histogram of the monthly temperatures in Seattle, with the color of the bars binned to indicate temperature. 
   3. Create a box and whiskers [plot](https://altair-viz.github.io/gallery/boxplot.html) of the monthly temperatures in Seattle (months on the x-axis)
   4. Create a markdown cell and discuss which idiom you like the most for viewing this data, and why

In [22]:
temps = data.seattle_temps.url

In [25]:
alt.Chart(temps).mark_area().encode(
    x='month(date):O',
    y='min(temp):Q',
    y2='max(temp):Q'
)

alt.Chart(...)

In [29]:
alt.Chart(temps).mark_bar().encode(
    y='month(date):O',
    color=alt.Color('mean(temp):Q', bin=True),
    x='count():Q'
)

alt.Chart(...)